<a href="https://colab.research.google.com/github/liam-haas/CLPS0950_FinalProject/blob/main/Project_Testbed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install rdkit-pypi

from rdkit import Chem
from rdkit.Chem import *
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import PandasTools
from rdkit.Chem import RDKFingerprint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 13.3 MB/s eta 0:00:00


In [4]:
import numpy as np

A = 'C=CC'
B = 'C=CC=C'

def index_fun(x):
  indices = [index for index in range(len(x)) if x.startswith('=',index)]
  # print(indices) # prints all occurances of double bonds in the SMILES string
  
  for n in range(0,len(indices)):

    a = indices[n] #testing to see if a phenyl is present

    
    if x[a-1] == 'C' and x[a+1] == 'C': # and x[a+6] != '1'
      

      print('Alkene found at position ', a)
    else:
      continue

  return indices

index_fun(A)

Alkene found at position  1


[1]

In [ ]:
import pandas as pd
# pd.read_csv('Test SMILES sequences - Sheet1.csv')
d = {'IUPAC name':['ethene','prop-1-ene','3-methylpent-1-ene','1-phenylprop-2-ene','but-1,3-diene'],
     'SMILES sequence':['C=C','C=CC','C=CC(CC)C','C=CCC1=CC=CC=C1','C=CC=C']}
d['Alkenes'] = [index_fun(item) for item in d['SMILES sequence']] 
df = pd.DataFrame(data=d)
df

**New strategy below:** converting SMILES string to list and searching for alkenes.

In [22]:
A = 'C=CC'
B = 'CC=CCC'
C = 'CC=C(CC)C'

def substitution(string): # determines the substition at each side of the alkene
  i = string.index('=')
  if string[i-1] == string [i+1]:
    if string[i-2] == string[i+2]:
      sub = 1  # 1 for equal substitution
    else:
      sub = 0 # 0 for unequal substition
  return sub

print(substitution(C))




################################################################################
def react(string):
  str_as_list = []
  str_as_list[:0] = string
  if '=' in str_as_list:
    i = A.index('=')

    del str_as_list[i:i+2]

    str_as_list.insert(i, 'C')
    str_as_list.insert(i+1, '(Br)')
    newMol = ''.join(str_as_list)
    pdt = Chem.MolFromSmiles(newMol)
    return pdt



def alkene_type(string):
  str_as_list = []
  str_as_list[:0] = string
  if '=' in str_as_list:
    i = A.index('=')

  try:
    # A[i-2], A[i-3], A[i-4], A[i+2], A[i+3], A[i+4]
    if (A[i-2] == 'C' and A[i-3] == 'C' and A[i-4] == 'C') or (A[i+2] == 'C' and A[i+3] == 'C' and A[i+4] == 'C'):
     alkene_type = 'tertiary'
  except IndexError:
    try: 
      # A[i-2], A[i-3], A[i+2], A[i+2]
      if (A[i-2] == 'C' and A[i-3] == 'C') or (A[i+2] == 'C' and A[i+2] == 'C'):
        alkene_type = 'secondary'
    except IndexError:
      try: 
        # A[i-2], A[i+2]
        if A[i-2] == [] or A[i+2] == []:
          alkene_type = 'primary'
      except IndexError:
        print('This is not a valid alkene')
        return alkene_type

0


In [13]:
from __future__ import print_function
from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolDescriptors

from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG=False

from itertools import chain

propene = Chem.MolFromSmiles('C=CC')
ethene = Chem.MolFromSmiles('C=C')
butene = Chem.MolFromSmiles('CC=CC')
mol1 = Chem.MolFromSmiles('C=CCC1=CC=CC=C1')

def substitution(string): # determines the substition at each side of the alkene
  i = string.index('=')
  if string[i-1] == string [i+1]:
    if string[i-2] == string[i+2]:
      sub = 1  # 1 for equal substitution
    else:
      sub = 0 # 0 for unequal substition
  return sub

def rxn1(x):
  a = Chem.MolFromSmiles(x)
  if substitution(x) == 0:
    rxn1 = AllChem.ReactionFromSmarts('[CH:1]=[C:2]>>[CH:1]([Br])[C:2]')
    pdt = rxn1.RunReactants((a, ))[0][0]
  else:
    rxn1 = AllChem.ReactionFromSmarts('') # change for equally substituted
    pdt = rxn1.RunReactants((a, ))[0][0]
    return pdt

rxn1('CCC=CC')

# import pandas as pd
# m = {'Alkenes':['C=C','C=CC','C=CCC','C/C=C/C','C=CCCC','C/C=C\C','C/C=C/CC','C/C=C\CC','C=CC(CC)C','C/C=C/C(C)C','C=C(C)C','C=CC(CC)CC','CCC(CC)/C=CC/C(C)C','C/C=C(CC)/CC','CCCC(CC)(C)/C=C/C']}
# m['Rxn with HBr'] = [rxn1(item) for item in m['Alkenes']]
# df = pd.DataFrame(data=m)
# df

# def rxn2(x):
#   rxn2 = AllChem.ReactionFromSmarts('[C:1]=[C:2][C1:3]>>[C:1][C:2]([Br])[C1:3]')
#   pdt = rxn2.RunReactants((mol1, ))[0][0]
#   return pdt

None


In [30]:
from itertools import combinations
import pandas as pd

features = ['C', '=', 'C', 'C']
tmp = []
for i in range(len(features)):
    oc = combinations(features, i + 1)
    for c in oc:
        tmp.append(list(c))
print(tmp)

d = {'TMPs':tmp}
d['Sequence'] = [''.join(a) for a in d['TMPs']]

def seq_to_struc(y):
  try: 
    [Chem.MolFromSmiles(b) for b in d['Sequence']]
  except TypeError: # we need to figure out how to predict this error
    print('N/A')
    return

d['Structure'] = [seq_to_struc(b) for b in d['Sequence']] # not working properly at the moment
df = pd.DataFrame(data=d)
df

n = {'Alkenes':['C=C', 'C=CC', 'C=CCC']}
n['Structure'] = [Chem.MolFromSmiles(z) for z in n['Alkenes']]
df = pd.DataFrame(data=n)
df

[['C'], ['='], ['C'], ['C'], ['C', '='], ['C', 'C'], ['C', 'C'], ['=', 'C'], ['=', 'C'], ['C', 'C'], ['C', '=', 'C'], ['C', '=', 'C'], ['C', 'C', 'C'], ['=', 'C', 'C'], ['C', '=', 'C', 'C']]


[14:25:54] SMILES Parse Error: syntax error while parsing: =
[14:25:54] SMILES Parse Error: Failed parsing SMILES '=' for input: '='
[14:25:54] SMILES Parse Error: syntax error while parsing: C=
[14:25:54] SMILES Parse Error: Failed parsing SMILES 'C=' for input: 'C='
[14:25:54] SMILES Parse Error: syntax error while parsing: =C
[14:25:54] SMILES Parse Error: Failed parsing SMILES '=C' for input: '=C'
[14:25:54] SMILES Parse Error: syntax error while parsing: =C
[14:25:54] SMILES Parse Error: Failed parsing SMILES '=C' for input: '=C'
[14:25:54] SMILES Parse Error: syntax error while parsing: =CC
[14:25:54] SMILES Parse Error: Failed parsing SMILES '=CC' for input: '=CC'
[14:25:54] SMILES Parse Error: syntax error while parsing: =
[14:25:54] SMILES Parse Error: Failed parsing SMILES '=' for input: '='
[14:25:54] SMILES Parse Error: syntax error while parsing: C=
[14:25:54] SMILES Parse Error: Failed parsing SMILES 'C=' for input: 'C='
[14:25:54] SMILES Parse Error: syntax error while p

,Alkenes,Structure
0,C=C,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f2260>
1,C=CC,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f21f0>
2,C=CCC,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f2730>


In [34]:
df

,Alkenes,Structure
0,C=C,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f2260>
1,C=CC,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f21f0>
2,C=CCC,<rdkit.Chem.rdchem.Mol object at 0x7f8e048f2730>
